In [1]:
import os

In [2]:
%pwd

'd:\\FutureExpertData\\Text-Summarizer-Project\\reseach'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\FutureExpertData\\Text-Summarizer-Project'

In [5]:
#entity
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class ModelTrainerConfig: 
    root_dir: Path 
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
   # evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [6]:
from textSummarizer.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from textSummarizer.utils.common import create_directories, read_yaml


class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH, 
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root]) #automatically create the folder that will contain the artefacts
        
        
    def get_model_trainer_config(self) -> ModelTrainerConfig: 
        config = self.config.model_trainer
        params = self.params.TrainingArguments
        
        create_directories([config.root_dir])
        
        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir, 
            data_path= config.data_path, 
            model_ckpt= config.model_ckpt, 
            num_train_epochs= params.num_train_epochs,
            warmup_steps= params.warmup_steps,
            per_device_train_batch_size= params.per_device_train_batch_size,
            weight_decay= params.weight_decay,
            logging_steps= params.logging_steps,
           # evaluation_strategy= params.evaluation_strategy, 
            eval_steps= params.eval_steps, 
            save_steps= params.save_steps, 
            gradient_accumulation_steps= params.gradient_accumulation_steps
        )   
        return model_trainer_config 
        

In [7]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

from datasets import load_dataset, load_from_disk
import torch

d:\FutureExpertData\Text-Summarizer-Project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from torch.utils.data import DataLoader
from transformers import Trainer

class CustomTrainer(Trainer):
    def get_train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.args.train_batch_size,
            shuffle=True,
            collate_fn=self.data_collator,
            pin_memory=False,  # 👈 Set to False here
        )

    def get_eval_dataloader(self, eval_dataset=None):
        eval_dataset = eval_dataset if eval_dataset is not None else self.eval_dataset
        return DataLoader(
            eval_dataset,
            batch_size=self.args.eval_batch_size,
            collate_fn=self.data_collator,
            pin_memory=False,  # 👈 Set to False here
        )


In [9]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

from datasets import load_dataset, load_from_disk
import torch

class ModelTrainer: 
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
        
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"

        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        dataset_samsum = load_from_disk(self.config.data_path)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)


        # Define training arguments using config values (from YAML)
        training_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=self.config.num_train_epochs,
            warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size,
            per_device_eval_batch_size=self.config.per_device_train_batch_size,  # or add eval batch param separately
            weight_decay=self.config.weight_decay,
            logging_steps=self.config.logging_steps,
           # evaluation_strategy=self.config.evaluation_strategy,
            eval_steps=self.config.eval_steps,
            save_steps=int(self.config.save_steps),  # just in case it's stored as float like 1e6
            gradient_accumulation_steps=self.config.gradient_accumulation_steps,
        )

        # Define the Trainer (you might need to adapt this to include tokenization logic)
        trainer = CustomTrainer(
        model=model_pegasus,
        args=training_args,
        tokenizer=tokenizer,
        data_collator=seq2seq_data_collator,
        train_dataset=dataset_samsum["test"].select(range(100)),
        eval_dataset=dataset_samsum["validation"].select(range(100)),

    )

        # Launch training
        trainer.train()
        
        #Save model$
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))
        
        #Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))


In [10]:
try: 
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config= model_trainer_config)
    model_trainer_config.train()
except Exception as e: 
    raise e

[2025-07-18 20:12:25,240: INFO: common: yaml file: config\config.yaml loaded successfully.]
[2025-07-18 20:12:25,240: INFO: common: yaml file: params.yaml loaded successfully.]
[2025-07-18 20:12:25,240: INFO: common: Created directory at: artifacts]
[2025-07-18 20:12:25,240: INFO: common: Created directory at: artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\chris.DESKTOP-GCU74DF\AppData\Local\Temp\ipykernel_24220\3072123999.py:39: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer = CustomTrainer(


ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`id` in this case) have excessive nesting (inputs type `list` where type `int` is expected).